In [ ]:
import nltk
#nltk.download('punkt')
import torch
import numpy as np
import datasets
from datasets import load_dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    AutoConfig
)

In [ ]:
# WARNING: long process (xsum, cnn)
#reddit = load_dataset("reddit", cache_dir="../cache")
#cnn = load_dataset("cnn_dailymail", "3.0.0", cache_dir="../cache")
#reddit_tifu = load_dataset("reddit_tifu", "long", cache_dir="../cache")
#samsum = load_dataset("samsum", cache_dir="../cache")
#cnn.save_to_disk("../data/cnn-dm")
#reddit_tifu.save_to_disk("../data/reddit-tifu")
#samsum.save_to_disk("../data/samsum")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large", cache_dir = "./cache")

In [27]:
samsum = load_from_disk("../data/samsum")
xsum = load_from_disk("../data/xsum")

In [53]:
# check sequence length
def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, return_length=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], return_length=True)

    model_inputs["labels"] = labels["input_ids"]

    print("samples:", len(model_inputs.length))
    print("source (avg)", sum(model_inputs.length)/len(model_inputs.length))
    print("target (avg)", sum(labels.length)/len(labels.length))
    print("source (min)", min(model_inputs.length), "(max)", max(model_inputs.length))
    print("target (min)", min(labels.length), "(max)", max(labels.length))

    return model_inputs

In [61]:
max_source_length = 512
max_target_length = 64

def truncate(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, return_overflowing_tokens=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, return_overflowing_tokens=True)

    model_inputs["labels"] = labels["input_ids"]

    print("source (truncated)", model_inputs["num_truncated_tokens"])
    print("target (truncated)", labels["num_truncated_tokens"])

    return model_inputs

In [64]:
sam = samsum["train"].map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

samples: 1000
source (avg) 157.728
target (avg) 27.438
source (min) 20 (max) 638
target (min) 7 (max) 75
samples: 1000
source (avg) 164.006
target (avg) 27.968
source (min) 17 (max) 703
target (min) 3 (max) 75
samples: 1000
source (avg) 154.585
target (avg) 27.289
source (min) 17 (max) 821
target (min) 6 (max) 90
samples: 1000
source (avg) 160.552
target (avg) 27.309
source (min) 21 (max) 773
target (min) 6 (max) 74
samples: 1000
source (avg) 158.821
target (avg) 27.719
source (min) 22 (max) 1081
target (min) 4 (max) 80
samples: 1000
source (avg) 152.994
target (avg) 26.85
source (min) 20 (max) 677
target (min) 6 (max) 73
samples: 1000
source (avg) 160.182
target (avg) 27.764
source (min) 2 (max) 869
target (min) 6 (max) 79
samples: 1000
source (avg) 150.226
target (avg) 27.297
source (min) 19 (max) 593
target (min) 5 (max) 83
samples: 1000
source (avg) 165.824
target (avg) 27.52
source (min) 21 (max) 4469
target (min) 6 (max) 73
samples: 1000
source (avg) 161.418
target (avg) 28.13
so

In [45]:
x = xsum["test"].map(preprocess_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

samples: 1000
source (avg) 499.319
target (avg) 28.096
source (min) 2 (max) 3175
target (min) 6 (max) 77
samples: 1000
source (avg) 509.236
target (avg) 27.97
source (min) 46 (max) 3174
target (min) 7 (max) 65
samples: 1000
source (avg) 488.139
target (avg) 28.197
source (min) 5 (max) 4531
target (min) 7 (max) 70
samples: 1000
source (avg) 493.184
target (avg) 28.07
source (min) 29 (max) 3696
target (min) 7 (max) 71
samples: 1000
source (avg) 483.254
target (avg) 28.012
source (min) 9 (max) 2463
target (min) 5 (max) 76
samples: 1000
source (avg) 488.888
target (avg) 28.083
source (min) 24 (max) 15278
target (min) 6 (max) 88
samples: 1000
source (avg) 500.922
target (avg) 28.123
source (min) 14 (max) 4095
target (min) 8 (max) 78
samples: 1000
source (avg) 489.854
target (avg) 28.51
source (min) 24 (max) 3379
target (min) 9 (max) 76
samples: 1000
source (avg) 496.445
target (avg) 28.162
source (min) 30 (max) 5596
target (min) 6 (max) 103
samples: 1000
source (avg) 479.006
target (avg) 27

In [ ]:
# reddit
# features: ['author', 'subreddit', 'summary', 'text']
# num_rows: 3848330
#reddit = reddit["train"].remove_columns(["id", "subreddit_id", "body", "normalizedBody"])
#reddit = reddit.rename_column("content", "text")
#reddit.save_to_disk("../data/reddit")
#reddit = load_from_disk("../data/reddit")
#reddit

In [ ]:
# reddit-dota
# features: ['author', 'summary', 'text']
# num_rows: 22366
#dota = reddit.filter(lambda x: x["subreddit"] == "DotA2")
#dota = dota.remove_columns("subreddit")

# bad text/summary @ dota[9]
#dota = dota.filter(lambda x: len(x["text"]) > len(x["summary"])) # source length > target length
#dota.flatten_indices().save_to_disk("../data/reddit-dota")
dota = load_from_disk("../data/reddit-dota")
dota

In [ ]:
# reddit-dota-proc
# num_rows: 19949
#max_text_length = 50000
min_summary_length = 20
max_ratio = 0.5
min_ratio = 0.005
#dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
proc = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
# bertscore filter?
proc.flatten_indices().save_to_disk("../data/reddit-dota-proc")
proc

In [ ]:
# reddit-dota-qtr
# num_rows: 14389
max_text_length = 3200
min_summary_length = 30
# 0.25(max: 128/512) & 0.125(max: 64/512)
max_ratio = 0.25
min_ratio = 0.01
dota = load_from_disk("../data/reddit-dota-proc")
dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
dota = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
dota.flatten_indices().save_to_disk("../data/reddit-dota-qtr")
dota = load_from_disk("../data/reddit-dota-qtr")
dota